In [ ]:
"""
The notebook was used for performing regressions & predictions in notebook. 
But now the regression parts moved to Stata.
So the notebook does not use any more.
"""

In [1]:
# add paths to sys path
from pathlib import Path
import sys
import numpy as np
import pandas as pd
import os

parent_dir = str(Path().resolve().parents[0])
sys.path.insert(0, parent_dir)
sys.path.insert(0, os.path.join(parent_dir, 'data_transformation'))

In [2]:
import statsmodels.api as sm
from sklearn.utils import resample
from scipy import stats

In [3]:
from config import *

from data_transformation.pipeline import (
    get_performance_credit_panel_data, 
    get_performance_credit_panel_data_2014, 
    get_performance_credit_relationship_panel_data
)

In [4]:
# Chinese
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns; sns.set()

%matplotlib inline

font_path = os.path.join(data_path, 'simsun.ttc')
prop = fm.FontProperties(fname=font_path)

plt.rcParams["font.sans-serif"]=["simsun"] 
plt.rcParams["axes.unicode_minus"]=False

In [541]:
# some varibles for X, y cols
control_cols = ['age', 'employee']

fe_cols= ['year', 'province_con', 'IndustryCode', 'organ_type_code']

credit_cols = ['red', 'green', 'black']

perf_origin_cols = ['TotalAssets', 'TotalLiability', 'IntangibleAsset',
                    'OperatingEvenue', 'OperatingCost', 'OperationProfit', 'NetProfit', 
                    'subsidy', 'patents_grant', 'RD']

perf_extend_cols = ['ROA', 'op_profit_margin', 'asset_turnover']

perf_growth_cols = ['net_profit_growth', 'op_profit_growth', 'sales_growth',
                       'op_cost_growth', 'assets_growth', 'liability_growth',
                       'in_assets_growth', 'asset_turnover_growth']

perf_cols =  perf_extend_cols + perf_origin_cols
perf_t0 = perf_extend_cols + ['TotalAssets', 'NetProfit'] # performance to control in t0 period



# ols2 {y: list of X features}
dict_credit_to_perf = {
    'TotalAssets': ['NetProfit', 'subsidy', 'patents_grant', 'RD', 'op_profit_margin'],
    'NetProfit': ['TotalAssets', 'subsidy', 'patents_grant', 'RD',
                'asset_turnover', 'debt_asset_ratio'],
    'subsidy': ['TotalAssets', 'NetProfit', 'patents_grant', 'RD'] + perf_extend_cols,
    'patents_grant': ['TotalAssets', 'NetProfit', 'subsidy', 'RD'] + perf_extend_cols,
    'RD': ['TotalAssets', 'NetProfit', 'subsidy', 'patents_grant'] + perf_extend_cols,
    'ROA': ['subsidy', 'patents_grant', 'RD', 'op_profit_margin', 'asset_turnover', 'debt_asset_ratio'],
    'op_profit_margin': ['TotalAssets', 'subsidy', 'patents_grant', 'RD'],
    'asset_turnover': ['NetProfit', 'subsidy', 'patents_grant', 'RD'],
    'debt_asset_ratio': ['NetProfit',  'subsidy', 'patents_grant', 'RD'],
    'net_profit_growth': ['TotalAssets', 'subsidy', 'patents_grant', 'RD',
                         'ROA', 'asset_turnover', 'debt_asset_ratio'],
    'sales_growth': ['TotalAssets', 'NetProfit', 'subsidy', 'patents_grant', 'RD',
                    'ROA', 'debt_asset_ratio'],
    #'op_profit_margin_growth': ['TotalAssets', 'TotalLiability', 'subsidy', 'patents_grant', 'RD',
    #                     'ROA', 'debt_asset_ratio'],
    'debt_asset_ratio_growth': ['NetProfit', 'subsidy', 'patents_grant', 'RD',
                               'op_profit_margin'],
    'asset_turnover_growth': ['NetProfit', 'subsidy', 'patents_grant', 'RD', 'ROA'],
    #'ROA_growth': ['TotalLiability', 'OperatingEvenue', 'subsidy', 'patents_grant', 'RD']
}

## regression level 1: performance -> credit

### regression level 1: Logistic regression

In [317]:
def get_ordered_regression_data(df_cp, balance=False):
    # data with all columns
    selected_cols = credit_cols + perf_cols + fe_cols
        
    data = df_cp.reset_index()[selected_cols]
    
    green_count = int(data.green.sum())
    
    # y -1, 0, 1
    # add FE for year, industry, and province 
    data = data.pipe(combine_credit_columns_into_one).pipe(handle_fe)

    if balance:
        data_red = resample(data[data.credit==1], replace=False, n_samples=green_count, random_state=42)
        data_green = data[data.credit==0]
        data_black = resample(data[data.credit==-1], replace=True, n_samples=green_count, random_state=42)
        
        data = pd.concat([data_red, data_green, data_black])
        
    # normalization
    data = data.pipe(normal_with_negative)
    
    return data

In [318]:
data = get_ordered_regression_data(df_cp, balance=False)

#Ordered logistic regression 
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

y = data.credit
X = data.drop('credit', axis=1)

estimator = OrderedModel(y, X)

result = estimator.fit(method='bfgs')

display(result.summary())

 1.0    13052
 0.0     9553
-1.0       96
Name: credit, dtype: int64
         Current function value: 0.445583
         Iterations: 500
         Function evaluations: 501
         Gradient evaluations: 501


X:\ProjectData\SCS\creditchina\env\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                 credit   Log-Likelihood:                -10115.
Model:                   OrderedModel   AIC:                         2.049e+04
Method:            Maximum Likelihood   BIC:                         2.151e+04
Date:                Wed, 05 Jul 2023                                         
Time:                        15:03:56                                         
No. Observations:               22701                                         
Df Residuals:                   22573                                         
Df Model:                         126                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
ROA                 -1.5088      0.514     -2.933      0.003      -2.517      -0.500
op_profit_margin     2.7251      0.338      8.066      0.000       2.063       3.387
asset_turnover       0.3707      0.136      2.733      0.006       0.105       0.636
debt_asset_ratio    -5.7251      0.430    -13.300      0.000      -6.569      -4.881
TotalAssets         -2.0073      0.436     -4.600      0.000      -2.863      -1.152
NetProfit            0.1185      0.026      4.488      0.000       0.067       0.170
subsidy              0.1237      0.082      1.512      0.131      -0.037       0.284
patents_grant        0.2406      0.118      2.034      0.042       0.009       0.472
RD                   0.3522      0.054      6.579      0.000       0.247       0.457
age                 -0.6944      0.121     -5.735      0.000      -0.932      -0.457
employee             2.4740      0.196     12.603      0.000       2.089       2.859
year_2014           -2.7469      0.051    -54.054      0.000      -2.847      -2.647
year_2015           -2.7340      0.049    -56.280      0.000      -2.829      -2.639
year_2016           -1.3605      0.038    -35.693      0.000      -1.435      -1.286
year_2017           -0.8250      0.037    -22.600      0.000      -0.897      -0.753
year_2018           -0.3513      0.037     -9.591      0.000      -0.423      -0.280
year_2019           -0.1623      0.037     -4.387      0.000      -0.235      -0.090
province_con_上海     -0.0545      0.072     -0.758      0.449      -0.196       0.087
province_con_云南     -0.8353      0.112     -7.428      0.000      -1.056      -0.615
province_con_内蒙古    -0.3623      0.133     -2.717      0.007      -0.624      -0.101
province_con_北京     -0.1781      0.071     -2.509      0.012      -0.317      -0.039
province_con_吉林     -0.2816      0.107     -2.624      0.009      -0.492      -0.071
province_con_四川     -0.3806      0.082     -4.665      0.000      -0.541      -0.221
province_con_天津     -0.0909      0.104     -0.878      0.380      -0.294       0.112
province_con_宁夏     -0.1103      0.168     -0.658      0.511      -0.439       0.218
province_con_安徽      0.1179      0.088      1.337      0.181      -0.055       0.291
province_con_山东      0.1724      0.077      2.237      0.025       0.021       0.323
province_con_山西     -0.3752      0.118     -3.182      0.001      -0.606      -0.144
province_con_广东     -0.2290      0.067     -3.418      0.001      -0.360      -0.098
province_con_广西     -0.2036      0.114     -1.785      0.074      -0.427       0.020
province_con_新疆     -0.1327      0.100     -1.324      0.186      -0.329       0.064
province_con_江苏     -0.2000      0.069     -2.887      0.004      -0.336      -0.064
province_con_江西      0.0613      0.109      0.563      0.573      -0.152       0.275
province_con_河北     -0.3196      0.099     -3.216      0.001      -0.514      -0.125
province_con_河南  

### regression level 1: OLS1 + Logit

In [331]:
def split_red_black(data, balance=True):
    green_count = int(data.green.sum())
    
    data_red = data[data.red==1]
    data_black = data[data.black==1]
    data_green = data[data.green==1]
    
    if balance:
        data_red = resample(data_red, replace=False, n_samples=green_count, random_state=42)
        data_black = resample(data_black, replace=True, n_samples=green_count, random_state=42)

    data_red = pd.concat([data_red, data_green])
    data_black = pd.concat([data_black, data_green])
    
    return data_red, data_black

In [569]:
def get_regression_data(df_cp, balance=False):
    # for ols1
    # data with all columns
    data = df_cp.reset_index()[credit_cols + control_cols + fe_cols + perf_cols + perf_growth_cols]
    
    # add FE for year, industry, and province          
    data = data[data.year > 2013].pipe(handle_fe)
    
    return data
        
    #data_red, data_black = split_red_black(data, balance)
    
    # normalization
    #data_red = data_red.pipe(normal_with_negative).pipe(remove_inf_data)
    #data_black = data_black.pipe(normal_with_negative).pipe(remove_inf_data)
    
    #return data_red, data_black

In [584]:
df_data = get_regression_data(df_cp)

In [583]:
def handle_fe(data):
    # add FE for year, industry, and province          
    data = pd.get_dummies(
        data, 
        columns=fe_cols[:-1],
        prefix=fe_cols[:-1],
        drop_first=True
    )
    
    # handle organ type(multi values) separately
    data = data.join(
        data.organ_type_code.str.get_dummies(sep=',').add_prefix('organ_') # drop the first column
    ).drop(['organ_type_code', 'organ_2', 'organ_3', 'IndustryCode_C39', 'province_con_浙江'], axis=1)  
    
    return data

In [612]:
# logit: 1 perfermance once
def logit_one_perf_to_credit(data):
    per_cols = perf_cols + perf_growth_cols
    
    #X_perf = data[per_cols]
    X_control_fe = data.drop(per_cols + credit_col, axis=1)
    #new_cols = [c for c in X_control_fe.columns if not c.startswith('Industry')]
    #X_control_fe = X_control_fe[new_cols]
    
    # check multicollinearlity
    for prefix in ['year', 'province', 'Industry', 'organ']:
        new_cols = [c for c in X_control_fe.columns if c.startswith(prefix)]
        ret = X_control_fe[new_cols].corr().abs().replace(1, 0).max()
        print(ret)
    
    # y
    y_red = data['red']
    y_black = data['black']
    
    def show_summary(credit_col, X, y):
        print(credit_col)        
        estimator = sm.OLS(y, sm.add_constant(X)).fit()
        print(estimator.summary())
    
    for x_per in per_cols:
        print(f'----------------{x_per}-------------------')
        
        # X
        X = data[x_per].to_frame().join(X_control_fe)
        print(X.shape)
        
        #show_summary('red', X, y_red)
        
        #show_summary('black', X, y_black)
        
    
logit_one_perf_to_credit(df_data)   

year_2015    0.172026
year_2016    0.181728
year_2017    0.194400
year_2018    0.197217
year_2019    0.203875
year_2020    0.203875
dtype: float64
province_con_上海     0.122973
province_con_云南     0.043249
province_con_内蒙古    0.035291
province_con_北京     0.131387
province_con_吉林     0.047564
province_con_四川     0.079323
province_con_天津     0.051455
province_con_宁夏     0.027002
province_con_安徽     0.073652
province_con_山东     0.102309
province_con_山西     0.043155
province_con_广东     0.144324
province_con_广西     0.043718
province_con_新疆     0.053262
province_con_江苏     0.144324
province_con_江西     0.049917
province_con_河北     0.054181
province_con_河南     0.065680
province_con_海南     0.041128
province_con_湖北     0.073594
province_con_湖南     0.072144
province_con_甘肃     0.041717
province_con_福建     0.081977
province_con_西藏     0.027298
province_con_贵州     0.035972
province_con_辽宁     0.062821
province_con_重庆     0.049917
province_con_陕西     0.050651
province_con_青海     0.022093
province_con

In [610]:
y

1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
        ... 
25112    0.0
25113    0.0
25114    0.0
25115    0.0
25116    0.0
Name: black, Length: 22702, dtype: float64

In [ ]:
x = df_data['province_con_重庆'].reset_index()
y = df_data.black

sm.Logit(y, X).fit().summary()

In [325]:
# performance -> credit
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error


def regression_perf_to_credit(data, credit_col, linear=True):    
    # credit_col: red or black
    y = data[credit_col].to_list()
    X = data.drop(credit_cols, axis=1)
    
    estimator = None
    
    if linear:
        estimator = sm.OLS(y, sm.add_constant(X)).fit()
    else:
        estimator = sm.Logit(y, sm.add_constant(X)).fit()#fit_regularized(method='l1', alpha=0.1)

    display(estimator.summary())

In [336]:
data_red, data_black = get_regression_data(df_cp, balance=False)

In [337]:
# ols1
regression_perf_to_credit(data_red, 'red', linear=True)

regression_perf_to_credit(data_black, 'black', linear=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     18.10
Date:                Wed, 05 Jul 2023   Prob (F-statistic):               0.00
Time:                        15:16:00   Log-Likelihood:                 9612.4
No. Observations:                9649   AIC:                        -1.898e+04
Df Residuals:                    9525   BIC:                        -1.809e+04
Df Model:                         123                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0100      0.051     -0.197      0.843      -0.110       0.090
ROA                  0.0157      0.047      0.338      0.735      -0.075       0.107
op_profit_margin    -0.3421      0.026    -13.010      0.000      -0.394      -0.291
asset_turnover      -0.0897      0.012     -7.597      0.000      -0.113      -0.067
debt_asset_ratio     0.7233      0.037     19.437      0.000       0.650       0.796
TotalAssets          0.0435      0.039      1.113      0.266      -0.033       0.120
NetProfit           -0.0002      0.002     -0.072      0.943      -0.005       0.004
subsidy             -0.0320      0.007     -4.474      0.000      -0.046      -0.018
patents_grant        0.0205      0.009      2.370      0.018       0.004       0.037
RD                  -0.0046      0.004     -1.070      0.285      -0.013       0.004
age                 -0.0166      0.011     -1.529      0.126      -0.038       0.005
employee            -0.0873      0.017     -5.071      0.000      -0.121      -0.054
year_2014           -0.0726      0.005    -15.148      0.000      -0.082      -0.063
year_2015           -0.0717      0.005    -15.340      0.000      -0.081      -0.063
year_2016           -0.0692      0.005    -14.625      0.000      -0.078      -0.060
year_2017           -0.0676      0.005    -13.856      0.000      -0.077      -0.058
year_2018           -0.0589      0.005    -11.418      0.000      -0.069      -0.049
year_2019           -0.0343      0.005     -6.387      0.000      -0.045      -0.024
province_con_上海      0.0116      0.007      1.636      0.102      -0.002       0.025
province_con_云南     -0.0010      0.010     -0.105      0.916      -0.020       0.018
province_con_内蒙古    -0.0117      0.012     -1.012      0.311      -0.034       0.011
province_con_北京      0.0123      0.007      1.758      0.079      -0.001       0.026
province_con_吉林      0.0286      0.010      2.893      0.004       0.009       0.048
province_con_四川      0.0077      0.008      1.000      0.317      -0.007       0.023
province_con_天津     -0.0026      0.010     -0.272      0.786      -0.022       0.016
province_con_宁夏     -0.0232      0.015     -1.524      0.128      -0.053       0.007
province_con_安徽      0.0146      0.009      1.674      0.094      -0.002       0.032
province_con_山东      0.0070      0.008      0.920      0.357      -0.008       0.022
province_con_山西     -0.0158      0.010     -1.514      0.130      -0.036       0.005
province_con_广东      0.0133      0.007      1.994      0.046       0.000       0.026
province_con_广西      0.0095      0.010      0.923      0.356      -0.011       0.030
province_con_新疆     -0.0151      0.009     -1.618      0.106      -0.033       0.003
province_con_江苏      0.0083      0.007      1.216      0.224      -0.005       0.022
province_con_江西      0.

In [ ]:
# TODO: LOGIT
regression_perf_to_credit(data_red, 'red', linear=False)

regression_perf_to_credit(data_black, 'black', linear=False)

In [ ]:
# TODO: balance

## regression level2: credit -> performance

In [340]:
def get_regression_perf_data(data, credit_into_one=False):
    # get data for regressions for credit->perf
    
    # performance columns
    df_perf = data.reset_index()[perf_cols].pipe(normal_with_negative)
    
    data = data.reset_index()[credit_cols + fe_cols].join(df_perf).pipe(handle_fe)
    
    # handle credit column
    # y -1, 0, 1
    
    if credit_into_one:
        data = data.pipe(combine_credit_columns_into_one)

    return data 
    

In [374]:
# auxilury functions for showing regression results
def get_regression_result_var():
    return {'r2':{}, 'reports': {}}

def add_regression_results(results, estimator, y_col):
    # add coef & p-value
    results['r2'][y_col] = round(estimator.rsquared, 3)
    results['reports'][y_col + '_coef'] = estimator.params
    results['reports'][y_col + '_pvalue'] = estimator.pvalues
    
    return results

def show_regression_results(results, credit_cols=['red', 'black']):
    for k, v in results['r2'].items():
        print(k, v)
    
    display(pd.DataFrame(results['reports']).loc[perf_cols + credit_cols, :].head(30).round(3))

In [376]:
def regression_credit_to_perf(data, credit_cols, show_report=False):
    data = data.reset_index(drop=True)
    
    y_cols = perf_extend_cols + perf_origin_cols
    
    results = get_regression_result_var()
    
    for y_col in y_cols:
        y = data[y_col]
        
        X = data.drop([y_col] + ['green'], axis=1)
        
        estimator = sm.OLS(y, sm.add_constant(X)).fit()
        
        results = add_regression_results(results, estimator, y_col)
        
        if show_report:
            print(f'\n\n---------------------{y_col}------------------')
            print(estimator.summary())
    
    # show results
    show_regression_results(results, credit_cols)
        

In [370]:
data = get_regression_perf_data(df_cp)

regression_credit_to_perf(data, credit_cols=['red', 'black'], show_report=False)

ROA 0.587
op_profit_margin 0.527
asset_turnover 0.418
debt_asset_ratio 0.493
TotalAssets 0.777
NetProfit 0.434
subsidy 0.326
patents_grant 0.432
RD 0.559


,ROA_coef,ROA_pvalue,op_profit_margin_coef,op_profit_margin_pvalue,asset_turnover_coef,asset_turnover_pvalue,debt_asset_ratio_coef,debt_asset_ratio_pvalue,TotalAssets_coef,TotalAssets_pvalue,NetProfit_coef,NetProfit_pvalue,subsidy_coef,subsidy_pvalue,patents_grant_coef,patents_grant_pvalue,RD_coef,RD_pvalue
ROA,NaN,NaN,0.647,0.000,0.456,0.000,-0.450,0.000,0.069,0.000,6.337,0.000,0.010,0.815,-0.042,0.181,-0.295,0.000
op_profit_margin,0.286,0.000,NaN,NaN,-0.071,0.000,-0.053,0.000,0.034,0.000,3.852,0.000,0.054,0.058,-0.056,0.007,-0.050,0.262
asset_turnover,0.028,0.000,-0.010,0.000,NaN,NaN,0.060,0.000,-0.099,0.000,0.437,0.000,0.020,0.057,0.004,0.581,0.119,0.000
debt_asset_ratio,-0.282,0.000,-0.075,0.000,0.617,0.000,NaN,NaN,0.238,0.000,-0.147,0.175,-0.024,0.472,-0.099,0.000,-0.665,0.000
TotalAssets,0.043,0.000,0.049,0.000,-1.007,0.000,0.238,0.000,NaN,NaN,1.157,0.000,0.728,0.000,0.405,0.000,0.475,0.000
NetProfit,0.015,0.000,0.021,0.000,0.017,0.000,-0.001,0.175,0.004,0.000,NaN,NaN,0.004,0.080,0.002,0.134,0.010,0.001
subsidy,0.000,0.815,0.003,0.058,0.008,0.057,-0.001,0.472,0.028,0.000,0.037,0.080,NaN,NaN,0.020,0.000,0.234,0.000
patents_grant,-0.002,0.181,-0.006,0.007,0.003,0.581,-0.007,0.000,0.029,0.000,0.043,0.134,0.037,0.000,NaN,NaN,0.199,0.000
RD,-0.003,0.000,-0.001,0.262,0.019,0.000,-0.011,0.000,0.008,0.000,0.044,0.001,0.096,0.000,0.045,0.000,NaN,NaN
age,-0.004,0.004,0.002,0.298,0.011,0.060,0.009,0.000,0.015,0.000,-0.111,0.000,-0.056,0.000,-0.006,0.398,-0.197,0.000


In [378]:
# consider 2 period of credit status
data1 = data.pipe(add_window_credit, credit_cols=['red', 'black'])

regression_credit_to_perf(
    data1, 
    credit_cols=['red', 'black', 'red_i-1', 'black_i-1'], 
    show_report=False)

ROA 0.587
op_profit_margin 0.527
asset_turnover 0.418
debt_asset_ratio 0.493
TotalAssets 0.777
NetProfit 0.435
subsidy 0.326
patents_grant 0.433
RD 0.559


,ROA_coef,ROA_pvalue,op_profit_margin_coef,op_profit_margin_pvalue,asset_turnover_coef,asset_turnover_pvalue,debt_asset_ratio_coef,debt_asset_ratio_pvalue,TotalAssets_coef,TotalAssets_pvalue,NetProfit_coef,NetProfit_pvalue,subsidy_coef,subsidy_pvalue,patents_grant_coef,patents_grant_pvalue,RD_coef,RD_pvalue
ROA,NaN,NaN,0.647,0.000,0.456,0.000,-0.450,0.000,0.070,0.000,6.330,0.000,0.010,0.806,-0.040,0.204,-0.298,0.000
op_profit_margin,0.286,0.000,NaN,NaN,-0.071,0.000,-0.053,0.000,0.034,0.000,3.851,0.000,0.054,0.057,-0.056,0.007,-0.051,0.251
asset_turnover,0.028,0.000,-0.010,0.000,NaN,NaN,0.060,0.000,-0.098,0.000,0.436,0.000,0.020,0.057,0.005,0.564,0.119,0.000
debt_asset_ratio,-0.282,0.000,-0.075,0.000,0.617,0.000,NaN,NaN,0.237,0.000,-0.143,0.187,-0.024,0.471,-0.101,0.000,-0.665,0.000
TotalAssets,0.044,0.000,0.049,0.000,-1.006,0.000,0.237,0.000,NaN,NaN,1.167,0.000,0.727,0.000,0.400,0.000,0.479,0.000
NetProfit,0.015,0.000,0.021,0.000,0.017,0.000,-0.001,0.187,0.004,0.000,NaN,NaN,0.004,0.079,0.002,0.112,0.010,0.002
subsidy,0.000,0.806,0.003,0.057,0.008,0.057,-0.001,0.471,0.028,0.000,0.037,0.079,NaN,NaN,0.020,0.000,0.234,0.000
patents_grant,-0.002,0.204,-0.006,0.007,0.003,0.564,-0.007,0.000,0.028,0.000,0.046,0.112,0.037,0.000,NaN,NaN,0.200,0.000
RD,-0.003,0.000,-0.001,0.251,0.019,0.000,-0.011,0.000,0.008,0.000,0.043,0.002,0.096,0.000,0.045,0.000,NaN,NaN
age,-0.004,0.006,0.002,0.301,0.011,0.055,0.009,0.000,0.014,0.000,-0.106,0.000,-0.056,0.000,-0.008,0.256,-0.195,0.000


In [379]:
# consider 2 period of credit status
v = data.pipe(add_window_credit, credit_cols=['red', 'black'], periods=2)

regression_credit_to_perf(
    data2, 
    credit_cols=['red', 'black', 'red_i-1', 'black_i-1', 'red_i-2', 'black_i-2'], 
    show_report=False)

ROA 0.588
op_profit_margin 0.527
asset_turnover 0.418
debt_asset_ratio 0.493
TotalAssets 0.778
NetProfit 0.435
subsidy 0.326
patents_grant 0.433
RD 0.559


,ROA_coef,ROA_pvalue,op_profit_margin_coef,op_profit_margin_pvalue,asset_turnover_coef,asset_turnover_pvalue,debt_asset_ratio_coef,debt_asset_ratio_pvalue,TotalAssets_coef,TotalAssets_pvalue,NetProfit_coef,NetProfit_pvalue,subsidy_coef,subsidy_pvalue,patents_grant_coef,patents_grant_pvalue,RD_coef,RD_pvalue
ROA,NaN,NaN,0.646,0.000,0.456,0.000,-0.449,0.000,0.071,0.000,6.326,0.000,0.011,0.804,-0.038,0.224,-0.301,0.000
op_profit_margin,0.286,0.000,NaN,NaN,-0.071,0.000,-0.053,0.000,0.035,0.000,3.850,0.000,0.054,0.057,-0.056,0.008,-0.051,0.245
asset_turnover,0.028,0.000,-0.010,0.000,NaN,NaN,0.060,0.000,-0.098,0.000,0.437,0.000,0.020,0.057,0.005,0.562,0.119,0.000
debt_asset_ratio,-0.282,0.000,-0.075,0.000,0.617,0.000,NaN,NaN,0.236,0.000,-0.141,0.192,-0.024,0.469,-0.101,0.000,-0.663,0.000
TotalAssets,0.045,0.000,0.049,0.000,-1.007,0.000,0.237,0.000,NaN,NaN,1.173,0.000,0.727,0.000,0.397,0.000,0.485,0.000
NetProfit,0.015,0.000,0.021,0.000,0.017,0.000,-0.001,0.192,0.004,0.000,NaN,NaN,0.004,0.078,0.002,0.110,0.010,0.002
subsidy,0.000,0.804,0.003,0.057,0.008,0.057,-0.001,0.469,0.028,0.000,0.038,0.078,NaN,NaN,0.020,0.000,0.234,0.000
patents_grant,-0.002,0.224,-0.006,0.008,0.003,0.562,-0.007,0.000,0.028,0.000,0.046,0.110,0.037,0.000,NaN,NaN,0.200,0.000
RD,-0.003,0.000,-0.001,0.245,0.019,0.000,-0.011,0.000,0.008,0.000,0.043,0.002,0.096,0.000,0.045,0.000,NaN,NaN
age,-0.004,0.012,0.002,0.275,0.011,0.057,0.009,0.000,0.013,0.000,-0.105,0.000,-0.056,0.000,-0.009,0.198,-0.193,0.000


## regression level2: credit -> growth performance

In [384]:
def get_regression_growth_data(df_cp, year_diff=1):
    data = df_cp.copy()
    
    # check year_diff in this function and in get_performance_data the same
    first_year = 2014 - year_diff
    
    if not first_year in data.index.get_level_values('year'):
        print('year_diff is inconsistent with the year_diff in df_cp, please check it.')
        return None
    
    # move perf to last period
    data[perf_t0] = data[
        perf_t0
    ].groupby(
        data.index.get_level_values('ID')
    ).shift(year_diff)
    
    data = data.groupby(
        data.index.get_level_values('ID'), as_index=False
    ).apply(lambda g: g.iloc[year_diff:, :])
    
    # for ols2 y=growth
    selected_cols = perf_cols + credit_cols + fe_cols + perf_growth_cols
        
    data = data.reset_index()[selected_cols].pipe(handle_fe).pipe(normal_with_negative)
    
    return data

In [394]:
def regression_credit_to_perf_growth(data, show_report=False):
    data = data.reset_index(drop=True)
    
    results = get_regression_result_var()
    
    for y_col in perf_growth_cols:
        y = data[y_col]
        X = data.drop(perf_growth_cols+['green'], axis=1)
        
        estimator = sm.OLS(y, sm.add_constant(X)).fit()
        
        # add coef & p-value
        add_regression_results(results, estimator, y_col)
        
        if show_report:
            print(f'\n\n---------------------{y_col}------------------')
            print(estimator.summary())
    
    show_regression_results(results)

In [395]:
data_growth = get_regression_growth_data(get_credit_performance_data(year_diff=1), year_diff=1)
regression_credit_to_perf_growth(data_growth)

net_profit_growth 0.242
sales_growth 0.095
debt_asset_ratio_growth 0.077
asset_turnover_growth 0.109
employment_growth 0.141


,net_profit_growth_coef,net_profit_growth_pvalue,sales_growth_coef,sales_growth_pvalue,debt_asset_ratio_growth_coef,debt_asset_ratio_growth_pvalue,asset_turnover_growth_coef,asset_turnover_growth_pvalue,employment_growth_coef,employment_growth_pvalue
ROA,-0.177,0.086,0.063,0.000,-0.084,0.000,0.042,0.000,0.172,0.000
op_profit_margin,-0.104,0.114,-0.046,0.000,-0.001,0.836,-0.102,0.000,-0.013,0.026
asset_turnover,0.134,0.000,-0.028,0.000,-0.004,0.014,-0.014,0.000,-0.040,0.000
debt_asset_ratio,-0.717,0.000,-0.001,0.815,-0.144,0.000,-0.007,0.002,-0.000,0.935
TotalAssets,-0.158,0.030,-0.107,0.000,-0.019,0.000,-0.002,0.539,-0.267,0.000
NetProfit,0.261,0.000,0.002,0.000,0.000,0.188,0.001,0.000,0.001,0.125
subsidy,0.057,0.000,0.004,0.000,-0.002,0.023,-0.001,0.180,0.004,0.002
patents_grant,0.004,0.826,-0.002,0.132,-0.001,0.715,-0.001,0.191,0.000,0.844
RD,-0.000,0.964,0.003,0.000,-0.001,0.391,-0.000,0.880,0.002,0.046
age,-0.082,0.000,-0.001,0.395,-0.004,0.016,0.000,0.813,-0.003,0.151


In [396]:
# year diff = 2
data_growth = get_regression_growth_data(get_credit_performance_data(year_diff=2), year_diff=2)
regression_credit_to_perf_growth(data_growth)

net_profit_growth 0.23
sales_growth 0.223
debt_asset_ratio_growth 0.127
asset_turnover_growth 0.05
employment_growth 0.23


,net_profit_growth_coef,net_profit_growth_pvalue,sales_growth_coef,sales_growth_pvalue,debt_asset_ratio_growth_coef,debt_asset_ratio_growth_pvalue,asset_turnover_growth_coef,asset_turnover_growth_pvalue,employment_growth_coef,employment_growth_pvalue
ROA,-0.416,0.000,0.059,0.000,-0.127,0.000,0.029,0.000,0.171,0.000
op_profit_margin,-0.157,0.001,-0.030,0.000,0.004,0.478,-0.046,0.000,0.018,0.012
asset_turnover,0.043,0.036,-0.059,0.000,-0.012,0.000,-0.016,0.000,-0.072,0.000
debt_asset_ratio,-0.621,0.000,0.027,0.000,-0.241,0.000,0.009,0.000,0.052,0.000
TotalAssets,-0.632,0.000,-0.252,0.000,-0.039,0.000,-0.009,0.002,-0.539,0.000
NetProfit,0.232,0.000,0.001,0.000,0.001,0.021,0.001,0.000,0.001,0.031
subsidy,0.026,0.048,0.010,0.000,-0.002,0.163,-0.000,0.817,0.010,0.000
patents_grant,0.029,0.102,0.002,0.204,-0.001,0.677,-0.000,0.796,0.004,0.158
RD,0.008,0.311,0.004,0.000,-0.001,0.226,-0.001,0.126,0.004,0.001
age,-0.027,0.172,-0.002,0.180,-0.007,0.001,0.001,0.535,-0.001,0.769


In [20]:
# with prediction
# performance -> credit
from linearmodels import PooledOLS
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error


def regression_perf_to_credit(data, credit_col, linear=True):    
    y = data[credit_col].to_list()
    X = data.drop(credit_cols, axis=1)
    
    """
    # remove columns whose sum < 2
    s_sum = X.sum()
    drop_cols = _sum[(s_sum==0) | (s_sum==1)].index.tolist()
    if len(drop_cols) > 0:
        X = X.drop(drop_cols, axis=1)
        print(drop_cols)
    """
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    # norm
    scalar = MinMaxScaler()
    X_train = pd.DataFrame(scalar.fit_transform(X_train), columns=X_train.columns)
    
    estimator = None
    if linear:
        estimator = sm.OLS(y_train, sm.add_constant(X_train)).fit()
    else:
        estimator = sm.Logit(y_train, sm.add_constant(X_train)).fit()#fit_regularized(method='l1', alpha=0.1)

    print(estimator.summary())
    
    # predict and check performance
    X_test = scalar.transform(X_test)
    y_predicted = estimator.predict(sm.add_constant(X_test))
    
    if linear:
        print(mean_squared_error(y_test, y_predicted))
    else:
        y_predicted = list(map(round, y_predicted))
    
        print(accuracy_score(y_test, y_predicted))
        print(confusion_matrix(y_test, y_predicted))
        print(classification_report(y_test, y_predicted))
    
    


In [24]:
# credit -> performance
y_cols = ['net_profit_growth', 'op_profit_growth',
       'sales_growth', 'op_profit_margin_growth', 'debt_asset_ratio_growth',
       'asset_turnover_growth', 'employment_growth', 'ROA_growth']

def regression_credit_to_perf(data, credit_col):
    data = data.reset_index(drop=True) # duplicate index in data_black
    
    for y_col in y_cols:
        print(f'\n\n---------------------{y_col}------------------')
        
        y = data[y_col]

        drop_cols = [c for c in credit_cols if c != credit_col] + [y_col]
        X = data.drop(drop_cols, axis=1)
        
        # split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # data scaling
        scalar = MinMaxScaler()
        data_train = X_train.join(y_train)
        data_train = pd.DataFrame(scalar.fit_transform(data_train), columns=data_train.columns)
        
        estimator = sm.OLS(data_train[y_col].tolist(), sm.add_constant(data_train.drop(y_col, axis=1))).fit()
        print(estimator.summary())

        # predict and check performance
        data_test = X_test.join(y_test)
        
        data_test = pd.DataFrame(scalar.transform(data_test), columns=data_test.columns)
        
        y_predicted = estimator.predict(sm.add_constant(data_test.drop(y_col, axis=1)))
        #y_predicted = list(map(round, y_predicted)) # for logistic regression

        print(mean_squared_error(y_test, y_predicted))


In [30]:
#RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

credit_col = 'black'
data = data_black

y = data[credit_col].to_list()
X = data.drop(credit_cols, axis=1)

# norm
scalar = MinMaxScaler()
X = pd.DataFrame(scalar.fit_transform(X), columns=X.columns)
    
rf_estimator = RandomForestRegressor(n_estimators=10)
rf_estimator.fit(X, y)

pd.DataFrame(rf_estimator.feature_importances_, index=X.columns.values).sort_values(by= 0, ascending=False)[:20]

,0
employment_growth,0.310179
year_2020,0.147370
ROA_growth,0.090463
subsidy,0.076642
patents_grant,0.062051
debt_asset_ratio_growth,0.056940
sales_growth,0.046488
IndustryCode_C15,0.039443
RD,0.033438
age,0.022902
